<a href="https://colab.research.google.com/github/shanto12/Baton_Master_Unity/blob/master/tencent_video_unrestricted_opus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Delete corrupted VAE
import os
import shutil

vae_path = "/content/drive/MyDrive/ComfyUI/ComfyUI/models/vae/hunyuan_video_vae_bf16.safetensors"
if os.path.exists(vae_path):
    os.remove(vae_path)
    print("✓ Deleted corrupted VAE")

tmp_path = vae_path + ".tmp"
if os.path.exists(tmp_path):
    os.remove(tmp_path)
    print("✓ Deleted tmp file")

print("Done! Now run the fixed script below.")

✓ Deleted corrupted VAE
Done! Now run the fixed script below.


In [6]:
# HunyuanVideo 1.5 - NSFW VIDEO GENERATOR
# Fixed: Reduced frames + more aggressive VAE tiling for OOM

import subprocess
import sys
import os
import time
import json
import urllib.request
import threading
import glob
from datetime import datetime
from base64 import b64encode
from IPython.display import HTML, display

# ============================================================
# SETTINGS - REDUCED FOR MEMORY
# ============================================================
VIDEO_PROMPT = """Beautiful woman with large natural breasts, completely nude, sensually touching herself,
erotic dancing, spreading legs, explicit full frontal nudity, intimate close-ups,
seductive eye contact, soft moaning expression, bedroom setting with silk sheets,
warm golden lighting, cinematic quality, photorealistic, 8K ultra HD,
smooth skin texture, perfect body proportions, professional adult film quality"""

# REDUCED SETTINGS to avoid OOM during VAE decode
WIDTH, HEIGHT = 544, 320
NUM_FRAMES = 25               # Reduced from 49 to 25 (~1 second)
STEPS = 30
GUIDANCE_SCALE = 7.0
FLOW_SHIFT = 9.0

# ============================================================
# LOGGING
# ============================================================
START_TIME = time.time()
SERVER_LOG = []

def log(msg, level="INFO"):
    elapsed = time.time() - START_TIME
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] [{elapsed:6.1f}s] [{level:5s}] {msg}")

def log_resources():
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used,memory.total,utilization.gpu',
                                 '--format=csv,noheader,nounits'], capture_output=True, text=True)
        if result.returncode == 0:
            parts = result.stdout.strip().split(', ')
            gpu_used, gpu_total, gpu_util = int(parts[0]), int(parts[1]), parts[2]
            gpu_pct = 100 * gpu_used / gpu_total

        with open('/proc/meminfo', 'r') as f:
            meminfo = f.read()
        mem_total = int([l for l in meminfo.split('\n') if 'MemTotal' in l][0].split()[1]) / 1024 / 1024
        mem_avail = int([l for l in meminfo.split('\n') if 'MemAvailable' in l][0].split()[1]) / 1024 / 1024
        mem_used = mem_total - mem_avail
        mem_pct = 100 * mem_used / mem_total

        log(f"📊 GPU: {gpu_used:,}/{gpu_total:,} MB ({gpu_pct:.0f}%) | Util: {gpu_util}% | RAM: {mem_used:.1f}/{mem_total:.1f} GB ({mem_pct:.0f}%)", "STATS")
    except:
        pass

print("="*70)
print("🔥 HUNYUANVIDEO 1.5 NSFW GENERATOR (Memory Optimized)")
print("="*70)
log(f"Prompt: {VIDEO_PROMPT[:80]}...")
log(f"Settings: {WIDTH}x{HEIGHT}, {NUM_FRAMES} frames (~{NUM_FRAMES/24:.1f}s), {STEPS} steps")
log_resources()
print("="*70)

# ============================================================
# STEP 1: DEPENDENCIES
# ============================================================
log("Checking dependencies...")

def check_deps():
    try:
        import torch, git, requests, websocket
        return True
    except:
        return False

if not check_deps():
    log("Installing dependencies...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q",
                    "torch==2.4.0", "torchaudio", "torchvision",
                    "--index-url", "https://download.pytorch.org/whl/cu121"], capture_output=True)
    for pkg in ["gitpython", "requests", "websocket-client", "psutil"]:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], capture_output=True)

log("✅ Dependencies ready")

import torch
import shutil
from google.colab import drive, runtime
from huggingface_hub import login

# ============================================================
# STEP 2: MOUNT DRIVE
# ============================================================
log("Mounting Google Drive...")

output_dir = '/content/drive/MyDrive/AI_Videos_Comfy'
work_space = '/content/drive/MyDrive/ComfyUI'

if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')

os.makedirs(output_dir, exist_ok=True)
os.makedirs(work_space, exist_ok=True)
log("✅ Drive mounted")

# ============================================================
# STEP 3: AUTH + COMFYUI
# ============================================================
log("Setting up ComfyUI...")

try:
    from google.colab import userdata
    login(token=userdata.get('HF_TOKEN'), add_to_git_credential=False)
    log("  Auth: HF token set")
except:
    pass

comfy_path = f'{work_space}/ComfyUI'
if not os.path.exists(comfy_path):
    log("  Cloning ComfyUI...")
    subprocess.run(["git", "clone", "--depth", "1", "https://github.com/comfyanonymous/ComfyUI", comfy_path],
                   capture_output=True, timeout=120)
else:
    log("  ComfyUI: exists")

os.chdir(comfy_path)
log("✅ ComfyUI ready")

# ============================================================
# STEP 4: CUSTOM NODES
# ============================================================
log("Setting up custom nodes...")

subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"],
               capture_output=True, timeout=120)

os.makedirs('custom_nodes', exist_ok=True)

nodes = [
    ("https://github.com/ltdrdata/ComfyUI-Manager", "ComfyUI-Manager"),
    ("https://github.com/kijai/ComfyUI-HunyuanVideoWrapper", "ComfyUI-HunyuanVideoWrapper"),
    ("https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite", "ComfyUI-VideoHelperSuite"),
]

for repo, name in nodes:
    path = f'custom_nodes/{name}'
    if not os.path.exists(path):
        subprocess.run(["git", "clone", "--depth", "1", repo, path], capture_output=True, timeout=60)
        log(f"  {name}: cloned")
    else:
        log(f"  {name}: exists")

for req in ['custom_nodes/ComfyUI-HunyuanVideoWrapper/requirements.txt',
            'custom_nodes/ComfyUI-VideoHelperSuite/requirements.txt']:
    if os.path.exists(req):
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", req], capture_output=True)

log("✅ Custom nodes ready")

# ============================================================
# STEP 5: CHECK MODELS
# ============================================================
log("Checking models...")

models_dir = 'models'
for d in ['vae', 'text_encoders', 'diffusion_models', 'clip', 'loras', 'LLM', 'hunyuan_video']:
    os.makedirs(f'{models_dir}/{d}', exist_ok=True)

os.makedirs('/content/model_downloads', exist_ok=True)

def download_model_fast(url, final_path, expected_size_gb=None):
    name = os.path.basename(final_path)

    if os.path.exists(final_path):
        size = os.path.getsize(final_path)
        size_gb = size / 1e9
        if expected_size_gb:
            if size_gb >= expected_size_gb * 0.95:
                return f"✓ {name} ({size_gb:.2f} GB)"
            else:
                log(f"  ⚠️ {name} size mismatch, re-downloading...")
                os.remove(final_path)
        else:
            return f"✓ {name} ({size_gb:.2f} GB)"

    local_path = f'/content/model_downloads/{name}'

    log(f"  ⬇️  Downloading {name}...")
    try:
        req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        with urllib.request.urlopen(req, timeout=1800) as response:
            total = int(response.headers.get('content-length', 0))
            downloaded = 0
            last_log = time.time()

            with open(local_path, 'wb') as f:
                while True:
                    chunk = response.read(8192*1024)
                    if not chunk:
                        break
                    f.write(chunk)
                    downloaded += len(chunk)

                    if total and time.time() - last_log > 10:
                        pct = 100 * downloaded / total
                        log(f"    {name}: {downloaded/1e9:.2f}/{total/1e9:.2f} GB ({pct:.0f}%)")
                        last_log = time.time()

        log(f"    Copying to Drive...")
        shutil.copy2(local_path, final_path)
        os.remove(local_path)

        final_size = os.path.getsize(final_path) / 1e9
        return f"✓ {name} ({final_size:.2f} GB)"

    except Exception as e:
        if os.path.exists(local_path):
            os.remove(local_path)
        return f"✗ {name}: {e}"

MODELS = [
    ("https://huggingface.co/Kijai/HunyuanVideo_comfy/resolve/main/hunyuan_video_vae_bf16.safetensors",
     f"{models_dir}/vae/hunyuan_video_vae_bf16.safetensors", 0.49),
    ("https://huggingface.co/Kijai/HunyuanVideo_comfy/resolve/main/hunyuan_video_720_cfgdistill_fp8_e4m3fn.safetensors",
     f"{models_dir}/diffusion_models/hunyuan_video_720_cfgdistill_fp8_e4m3fn.safetensors", 13.19),
]

for url, path, expected_size in MODELS:
    result = download_model_fast(url, path, expected_size)
    log(f"  {result}")

log("Checking text encoders...")

llm_path = f"{models_dir}/LLM/llava-llama-3-8b-text-encoder-tokenizer"
if os.path.exists(llm_path):
    shards = [f"model-0000{i}-of-00004.safetensors" for i in range(1, 5)]
    missing = [s for s in shards if not os.path.exists(os.path.join(llm_path, s))]
    if missing:
        log(f"  ⚠️ LLM incomplete, removing...")
        shutil.rmtree(llm_path)
    else:
        total_size = sum(os.path.getsize(os.path.join(llm_path, s)) for s in shards if os.path.exists(os.path.join(llm_path, s)))
        log(f"  ✓ LLM text encoder OK ({total_size/1e9:.1f}GB)")
else:
    log(f"  LLM: will auto-download")

clip_path = f"{models_dir}/clip/clip-vit-large-patch14"
if os.path.exists(clip_path) and os.path.exists(os.path.join(clip_path, "model.safetensors")):
    size = os.path.getsize(os.path.join(clip_path, "model.safetensors"))
    log(f"  ✓ CLIP OK ({size/1e9:.2f}GB)")
else:
    log(f"  CLIP: will auto-download")

log("✅ Models ready")
log_resources()

# ============================================================
# STEP 6: START SERVER
# ============================================================
log("Starting ComfyUI server...")

subprocess.run(["pkill", "-9", "-f", "main.py"], capture_output=True)
subprocess.run(["pkill", "-9", "-f", "comfyui"], capture_output=True)
time.sleep(3)

import socket
def is_port_free(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        try:
            s.bind(('127.0.0.1', port))
            return True
        except:
            return False

port = 8188
if not is_port_free(port):
    subprocess.run(["fuser", "-k", f"{port}/tcp"], capture_output=True)
    time.sleep(2)
    if not is_port_free(port):
        port = 8189

log(f"  Starting server on port {port}...")
server_process = subprocess.Popen(
    ["python", "main.py", "--listen", "127.0.0.1", "--port", str(port),
     "--output-directory", output_dir, "--disable-auto-launch"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

def read_server_output():
    for line in server_process.stdout:
        line = line.strip()
        if line:
            SERVER_LOG.append(line)

output_thread = threading.Thread(target=read_server_output, daemon=True)
output_thread.start()

server_address = f"127.0.0.1:{port}"
log(f"  Waiting for server...")

server_ready = False
for i in range(400):
    if server_process.poll() is not None:
        log(f"❌ Server died!", "ERROR")
        for line in SERVER_LOG[-20:]:
            log(f"   {line[:100]}", "ERROR")
        break

    try:
        urllib.request.urlopen(f"http://{server_address}/system_stats", timeout=2)
        log(f"✅ Server ready ({i}s)")
        server_ready = True
        break
    except:
        if i % 30 == 0 and i > 0:
            log(f"  Still loading... ({i}s)")
        time.sleep(1)

if not server_ready:
    raise Exception("Server failed to start")

# ============================================================
# STEP 7: DISCOVER NODES
# ============================================================
time.sleep(3)
log("Discovering nodes...")

with urllib.request.urlopen(f"http://{server_address}/object_info", timeout=60) as r:
    available_nodes = json.loads(r.read())

hy_count = len([k for k in available_nodes if 'hyvideo' in k.lower() or 'hunyuan' in k.lower()])
log(f"✅ Found {hy_count} HunyuanVideo nodes")

scheduler_options = []
if 'HyVideoSampler' in available_nodes:
    sampler_info = available_nodes['HyVideoSampler']
    if 'input' in sampler_info and 'required' in sampler_info['input']:
        for param, info in sampler_info['input']['required'].items():
            if param == 'scheduler' and isinstance(info, list) and len(info) > 0:
                scheduler_options = info[0] if isinstance(info[0], list) else []

SCHEDULER = scheduler_options[0] if scheduler_options else "FlowMatchDiscreteScheduler"
log(f"  Using scheduler: {SCHEDULER}")

# ============================================================
# STEP 8: BUILD WORKFLOW (Memory Optimized)
# ============================================================
log("Building workflow...")

import random
SEED = random.randint(0, 2**63-1)
clean_prompt = " ".join(VIDEO_PROMPT.split())

workflow = {
    "1": {
        "class_type": "HyVideoModelLoader",
        "inputs": {
            "model": "hunyuan_video_720_cfgdistill_fp8_e4m3fn.safetensors",
            "base_precision": "bf16",
            "quantization": "disabled",
            "load_device": "main_device",
            "attention_mode": "sdpa",
        }
    },
    "2": {
        "class_type": "HyVideoVAELoader",
        "inputs": {
            "model_name": "hunyuan_video_vae_bf16.safetensors",
            "precision": "bf16",
        }
    },
    "10": {
        "class_type": "DownloadAndLoadHyVideoTextEncoder",
        "inputs": {
            "llm_model": "Kijai/llava-llama-3-8b-text-encoder-tokenizer",
            "clip_model": "openai/clip-vit-large-patch14",
            "precision": "bf16",
        }
    },
    "3": {
        "class_type": "HyVideoTextEncode",
        "inputs": {
            "text_encoders": ["10", 0],
            "prompt": clean_prompt,
            "force_offload": True,
        }
    },
    "4": {
        "class_type": "HyVideoSampler",
        "inputs": {
            "model": ["1", 0],
            "hyvid_embeds": ["3", 0],
            "width": WIDTH,
            "height": HEIGHT,
            "num_frames": NUM_FRAMES,
            "steps": STEPS,
            "embedded_guidance_scale": GUIDANCE_SCALE,
            "flow_shift": FLOW_SHIFT,
            "seed": SEED,
            "scheduler": SCHEDULER,
            "force_offload": True,
            "denoise_strength": 1.0,
        }
    },
    "7": {
        "class_type": "HyVideoDecode",
        "inputs": {
            "vae": ["2", 0],
            "samples": ["4", 0],
            "enable_vae_tiling": True,
            "auto_tile_size": False,  # Manual tile size for better memory control
            "spatial_tile_sample_min_size": 128,  # Smaller tiles (was 256)
            "temporal_tiling_sample_size": 16,    # Smaller temporal tiles (was 64)
        }
    },
    "8": {
        "class_type": "VHS_VideoCombine",
        "inputs": {
            "images": ["7", 0],
            "frame_rate": 24,
            "loop_count": 0,
            "filename_prefix": "HunyuanVideo_NSFW",
            "format": "video/h264-mp4",
            "save_output": True,
            "pingpong": False,
            "pix_fmt": "yuv420p",
        }
    }
}

log(f"✅ Workflow ready (seed: {SEED})")

# ============================================================
# STEP 9: GENERATE
# ============================================================
print("\n" + "="*70)
log("🔥 STARTING NSFW VIDEO GENERATION")
print("="*70)
log(f"Prompt: {clean_prompt[:100]}...")
log(f"Resolution: {WIDTH}x{HEIGHT}, {NUM_FRAMES} frames (~{NUM_FRAMES/24:.1f}s), {STEPS} steps")
log_resources()

existing_videos = set(glob.glob(f"{output_dir}/**/*.mp4", recursive=True))

log("Queueing prompt...")
try:
    data = json.dumps({"prompt": workflow}).encode()
    req = urllib.request.Request(f"http://{server_address}/prompt", data=data)
    req.add_header('Content-Type', 'application/json')
    response = urllib.request.urlopen(req, timeout=30)
    result = json.loads(response.read())
    prompt_id = result['prompt_id']
    log(f"✅ Queued: {prompt_id}")
except urllib.error.HTTPError as e:
    err = json.loads(e.read().decode())
    log(f"❌ Queue failed: {err}", "ERROR")
    raise

log("⏳ Generating (5-20 min on A100)...")
gen_start = time.time()
last_log = 0

def get_history(pid):
    try:
        with urllib.request.urlopen(f"http://{server_address}/history/{pid}", timeout=10) as r:
            return json.loads(r.read())
    except:
        return {}

def get_queue():
    try:
        with urllib.request.urlopen(f"http://{server_address}/queue", timeout=10) as r:
            return json.loads(r.read())
    except:
        return {}

while True:
    time.sleep(5)
    elapsed = time.time() - gen_start

    if elapsed - last_log >= 30:
        queue = get_queue()
        running = len(queue.get('queue_running', []))
        log(f"⏳ {elapsed/60:.1f} min | Queue: {running} running")
        log_resources()

        progress = [l for l in SERVER_LOG[-20:] if any(x in l.lower() for x in
            ['sampling', '%', 'step', 'diffusion', 'it/s', 'loading', 'decod', 'vae'])]
        for line in progress[-2:]:
            log(f"   [SERVER] {line[:80]}", "DEBUG")
        last_log = elapsed

    history = get_history(prompt_id)
    if prompt_id in history:
        if history[prompt_id].get('outputs'):
            log("✅ Generation complete!")
            break
        status = history[prompt_id].get('status', {})
        if status.get('status_str') == 'error':
            log(f"❌ Error!", "ERROR")
            for line in SERVER_LOG[-30:]:
                if 'error' in line.lower() or 'exception' in line.lower() or 'oom' in line.lower():
                    log(f"   {line[:100]}", "ERROR")
            break

    queue = get_queue()
    if elapsed > 180 and not queue.get('queue_running') and not queue.get('queue_pending'):
        if get_history(prompt_id).get(prompt_id, {}).get('outputs'):
            break
        errors = [l for l in SERVER_LOG[-30:] if 'error' in l.lower() or 'exception' in l.lower()]
        if errors:
            log("❌ Failed", "ERROR")
            for e in errors[-5:]:
                log(f"   {e[:100]}", "ERROR")
            break

    if elapsed > 3600:
        log("❌ Timeout!", "ERROR")
        break

gen_time = time.time() - gen_start

# ============================================================
# STEP 10: FIND AND PLAY VIDEO
# ============================================================
log("Looking for video...")

video_path = None

history = get_history(prompt_id)
if prompt_id in history:
    for node_output in history[prompt_id].get('outputs', {}).values():
        for key in ['gifs', 'videos', 'images']:
            if key in node_output:
                for item in node_output[key]:
                    fn = item.get('filename', '')
                    for try_path in [os.path.join(output_dir, fn), os.path.join(f'{work_space}/ComfyUI/output', fn)]:
                        if os.path.exists(try_path):
                            video_path = try_path
                            break

if not video_path:
    for d in [output_dir, f'{work_space}/ComfyUI/output']:
        if os.path.exists(d):
            for f in glob.glob(os.path.join(d, '**/*.mp4'), recursive=True):
                if f not in existing_videos:
                    video_path = f
                    break

if video_path and os.path.exists(video_path):
    size = os.path.getsize(video_path) / 1e6
    log(f"✅ Video: {video_path} ({size:.1f} MB)")

    try:
        with open(video_path, "rb") as f:
            video_b64 = b64encode(f.read()).decode()

        display(HTML(f'''
        <div style="background:#1a1a2e;padding:20px;border-radius:10px;margin:10px 0;">
            <h2 style="color:#ff6b6b;">🔥 NSFW Video Generated!</h2>
            <video width="720" controls autoplay loop style="border-radius:8px;border:2px solid #ff6b6b;">
                <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
            </video>
            <p style="color:#aaa;margin-top:10px;">
                📁 {video_path}<br>
                📊 {size:.1f} MB | ⏱️ {gen_time/60:.1f} min | 🎬 {NUM_FRAMES} frames (~{NUM_FRAMES/24:.1f}s)
            </p>
        </div>
        '''))
        log("✅ Video displayed!")
    except Exception as e:
        log(f"Display error: {e}", "WARN")
else:
    log("❌ No video found!", "ERROR")
    for line in SERVER_LOG[-30:]:
        log(f"   {line[:100]}", "DEBUG")

# ============================================================
# CLEANUP
# ============================================================
print("\n" + "="*70)
log("🔥 DONE!" if video_path else "⚠️ ISSUES")
print("="*70)
log(f"Total: {(time.time()-START_TIME)/60:.1f} min | Gen: {gen_time/60:.1f} min")

if video_path:
    log("🔌 Disconnecting in 30s...")
    time.sleep(30)
    server_process.terminate()
    runtime.unassign()
else:
    log("⚠️ Not disconnecting - check errors")

🔥 HUNYUANVIDEO 1.5 NSFW GENERATOR (Memory Optimized)
[01:35:19] [   0.0s] [INFO ] Prompt: Beautiful woman with large natural breasts, completely nude, sensually touching ...
[01:35:19] [   0.0s] [INFO ] Settings: 544x320, 25 frames (~1.0s), 30 steps
[01:35:19] [   0.0s] [STATS] 📊 GPU: 1,123/40,960 MB (3%) | Util: 0% | RAM: 42.8/83.5 GB (51%)
[01:35:19] [   0.0s] [INFO ] Checking dependencies...
[01:35:19] [   0.0s] [INFO ] ✅ Dependencies ready
[01:35:19] [   0.0s] [INFO ] Mounting Google Drive...
[01:35:19] [   0.0s] [INFO ] ✅ Drive mounted
[01:35:19] [   0.0s] [INFO ] Setting up ComfyUI...
[01:35:19] [   0.3s] [INFO ]   Auth: HF token set
[01:35:19] [   0.3s] [INFO ]   ComfyUI: exists
[01:35:19] [   0.3s] [INFO ] ✅ ComfyUI ready
[01:35:19] [   0.3s] [INFO ] Setting up custom nodes...
[01:35:22] [   3.4s] [INFO ]   ComfyUI-Manager: exists
[01:35:22] [   3.4s] [INFO ]   ComfyUI-HunyuanVideoWrapper: exists
[01:35:22] [   3.4s] [INFO ]   ComfyUI-VideoHelperSuite: exists
[01:35:28] [   9.0

[01:39:06] [ 227.4s] [INFO ] ✅ Video displayed!

[01:39:06] [ 227.4s] [INFO ] 🔥 DONE!
[01:39:06] [ 227.4s] [INFO ] Total: 3.8 min | Gen: 3.1 min
[01:39:06] [ 227.4s] [INFO ] 🔌 Disconnecting in 30s...
